# Flux measurements training SFERA-III STT16
![](Figure\\sfera3.png)
![](Figure\\promes_haute_resolution.jpg)
![](Figure\\cnrs.png)

# General function module
In this module the different functions use to obtain fluxmetry cartographie are defined.

## Modules
Here the modules use in the functions defined below are imported.

In [1]:
# System interphase
import os
# Interface system with user
import tkinter
# Date and time management
import datetime
import pytz
# Graphic
import matplotlib.pyplot as plt

# Matrice management
import numpy as np
# URL handling
import urllib

## Constances

In [ ]:
# Value of the DNI use to normalize the different physical values
NORM_DNI = 1000 # (W/m²)
# Isobaric mass heat capacity at 25 °C
# ref : https://en.wikipedia.org/wiki/Table_of_specific_heat_capacities
Cp_H2O = 4185 * 10**-3  # (J/g/K)

# Surface of the calorimeter diaphragme
DIAPH_SURF = (10e-3/2)**2 * np.pi #(m²)

## Fluxmetry map
In this section, the functions use in the `ex_flux_map` file are defined.

### Function `get_img_parameters`
This function reads the log file obtain with the PETALES2 sowftware and return the relevant parameters

In [4]:
def get_img_parameters(log_path) :
    """
    Get the relevant parameters in the log file
    """
    #### Date and time ####
    log_name = os.path.basename(log_path)
    date, time = get_log_time(log_name)
    
    ### Intitialization ###
    # List of interst parameters
    parameter_names =["ExposureValue", 
                      "Gamma",
                      "WhitebalValueRed",
                      "WhitebalValueBlue",
                      "GainValue",
                      "OffsetValue",
                      "TimeStampFrequency",
                      "CameraName",
                      "DeviceSerialNumber",
                      "DeviceModelName"]
    # Create a dictionary for storing parameters
    parameters = {"Date": date,
                  "Time": time}
    
    #### Log file management #### 
    log_file = open(log_path, "r")
    content = log_file.readlines() 
    # Browse the lines in the log file
    for ligne in content:  
        # Is the line corresponds to a parameter to be saved ?
        if is_param_in_line(ligne, parameter_names) :  
            # then a new instance is created in the dictionary
            value = get_value(ligne) 
            # the value is saved
            parameters[ligne[7:ligne.find("=")]] = value
    ## Display a warning window when values do not match default values        
    check_parameters(parameters)
    return parameters


def is_param_in_line(ligne, parameter_names):
    """
    Is the line corresponds to a parameter to be saved ? 
    """
    test =[]
    for parametre in parameter_names :
        test.append(ligne[7:7+len(parametre)] ==  parametre)
    return any(test)


def get_value(ligne) :
    """
    Get the parameter value in the line 
    """
    value = ligne[ligne.find('=')+1:]
    try :
        # Convert in float
        value = float(value)  
    except :
        value = value[:-1]
    return value


def check_parameters(parameters):
    """
    Checks that the acquisition parameters used are the good ones. 
    If not, raise a warnig window.
    """
    good_parameter = {"WhitebalValueRed" : 120, "WhitebalValueBlue":224, "GainValue":0, "OffsetValue":0, "Gamma":1 }
    test = [parameters[parameter] != good_parameter[parameter] for parameter in good_parameter]
    if any(test) :
         display_warning_win(parameters, good_parameter)  # Affichage d'un fenetre d'avertissement si les paramètres d'acquisition ne correspondent pas à ceux attendus

def display_warning_win(used_parameters, good_parameter):
    """
    Display a warning windoyw with the wrong parameter highlighted
    """
    # Create a window
    win= tkinter.Tk()
    # Define the siza
    win.geometry("600x200")
    # place the window in front of any of the user's other windows
    win.attributes('-topmost', 1)
    # Creates a frame for buttons
    frame_value = tkinter.Frame(win)
    # Set the frame in the bottom of the window
    frame_value.pack(side = tkinter.BOTTOM)
    # Create a label
    text=u"Warning at least one of the parameter values does not correspond to the correct value"
    label = tkinter.Label(win, text=text, font=('Arial 12 bold'))
    # Display label
    label.pack(side = tkinter.TOP)
    
    # For each parameter, display the value used and the good value
    for parametre in good_parameter.keys() :
        used_value = used_parameters[parametre]
        good_value = good_parameter[parametre]
        text = parametre + "=" + str(used_value) + ", valeur attendue = " + str(good_value)
        police = set_font(used_value, good_value)
        # Create a label
        label = tkinter.Label(frame_value, text=text, font=(police), compound="left") # Crée une étiquette
         # Display label
        label.pack()
    # Display window
    win.mainloop()

    
def set_font(used_value, good_value) :
    """
    Set the font to use
    If the used value is not the good value, then the font is bold
    """
    if used_value == good_value :
        police = 'Arial 11'
    else :
        police = 'Arial 11 bold'
    return police

def get_log_time(log_name) :
    date = '20' + log_name[11:19]
    time = log_name[20:22] + '-' + log_name[22:24] + '-' + log_name[24:26]
    return date, time


### Function `get_DNI`
This function return the Direct Normal Irriadiation value (W/m²) at the instant entered in argument

In [5]:
def get_DNI(date, time) :
    """
    This function return the Direct Normal Irriadiation value (W/m²) at the instant entered in argument
    Input : 
        date as "YYYY-MM-DD"
        time as [HH, MM, SS]
    Output : 
        DNI value (float)
    """
    summer_time = is_summer_time(date)
    hour = str(float(time[0]) - 1 - 1 * summer_time)
    minute = str(time[1])
    second = str(time[2])
    time = hour + ":" + minute + ":" + second
    url = "http://193.50.5.248/donneesMeteo/getData.php?quand=" \
           + time + "&jour=" + date + "&qui=3"
    response = urllib.request.urlopen(url)
    html = response.read()
    DNI = float(html)
    return DNI

### Function `is_summer_time`
This function return if a date corresponds to summertime 

In [6]:
def is_summer_time(date):
    """ 
    Test if a date corresponds to summer time
    Input : date (YYYY-MM-DD)
    Output : bool
    """
    year = int(date[0:4])
    month = int(date[5:7])
    day = int(date[8::])
    # Date to test
    date_Paris = datetime.datetime(year, month, day, hour=12)
    #  Set Paris timezone
    pytz.timezone("Europe/Paris").localize(date_Paris, is_dst=None)
    #  Convert UTC
    date_UTC = date_Paris.astimezone(pytz.utc)
    #  (heure) Difference between UTC and Paris
    diff_Paris_UTC = date_Paris.hour - date_UTC.hour
    # (booléen) Test if date belongs to summer time,
    # i.e in summer Paris time = UTC +2
    test = diff_Paris_UTC == 2
    return test

## Radiometric calibration
In this section, the functions use in the radiometric calibration are defined.

### Functions `read_value`
These functions are used to read the txt file from the calorimeter.

In [3]:
def read_value(function):
    def wrapper(line):
        """
        This function return the interest element on a line
        """
        col_num = {'heure': 0,
           'DNI': 2,
           'T1': 3,
           'T2': 4,
           'Q': 5}
        col_idx = function(line, col_num)
        caract_separation = [idx for idx in range(len(line))
                             if line[idx] == "	"]
        caract_separation.insert(0, 0)
        value = line[caract_separation[col_idx]:caract_separation[col_idx+1]]
        return value
    return wrapper

@read_value
def read_time(line, col_num):
    col_idx = col_num["heure"]
    return col_idx

@read_value
def read_DNI(line, col_num):
    col_idx = col_num["DNI"]
    return col_idx

@read_value
def read_T1(line, col_num):
    col_idx = col_num["T1"]
    return col_idx

@read_value
def read_T2(line, col_num):
    col_idx = col_num["T2"]
    return col_idx

@read_value
def read_flow(line, col_num):
    col_idx = col_num["Q"]
    return col_idx

## Spatial calibration
In this section, the functions use in the `spatial_calibration` file are defined.

### Functions `order_along_x` and `order_along_y`:
These functions order the circles along one axis.

In [ ]:
def order_along_y(circles):
    """
    Order the circles along the y coordinate
    # Input ordianary array
    # Output ordianary array
    """
    order = np.argsort(circles, axis=0, kind="quicksort", order=None)[:, 0]
    circles = circles[order, :]
    return circles

In [9]:
def order_along_x(circles):
    """
    Order the circles along the x coordinate
    # Input ordianary array
    # Output ordianary array
    """
    order = np.argsort(circles, axis=0, kind="quicksort", order=None)[:, 1]
    circles = circles[order, :]
    return circles

### Function `calculate_distances_between_point_sets`
Calculates distances between two sets of two-dimensional points.


In [2]:
def calculate_distances_between_point_sets(points1, points2):
    """
    Calculates distances between two sets of two-dimensional points.

    Args:
        point_set1 (array-like): The first set of points in [x, y] format.
        point_set2 (array-like): The second set of points in [x, y] format.

    Returns:
        array-like: An array of the distances between each pair of points in the two sets.
    """
    
    x1, y1 = points1[:, :, 0], points1[:, :, 1]
    x2, y2 = points2[:, :, 0], points2[:, :, 1]
    distance = np.sqrt((x1-x2)**2 + (y1-y2)**2)
    return distance

In [51]:
def plot_mean_distortion(dist, orientation, pixel_nb):
    if orientation=="h":
        axis = 0
        marker = r"$\leftrightarrow$"
        axis_title = "Average of the horizontal deviation"
    elif orientation == "v":
        axis = 1
        marker =  r"$\updownarrow$"
        axis_title = "Average of the vertical deviation"
    cm = plt.cm.get_cmap('RdYlBu')
    mean_bary = np.mean(dist[1][axis], axis=axis)
    mean_dist = np.mean(dist[0], axis=axis) - pixel_nb

    if orientation=="h":
        plt.scatter(mean_bary, mean_dist, c=mean_dist,
                    vmin=-1, vmax=1, cmap=cm,
                    marker=marker, s=100)
        plt.ylabel(axis_title)

    elif orientation == "v":
        plt.scatter(mean_dist, mean_bary, c=mean_dist,
                    vmin=-1, vmax=1, cmap=cm,
                    marker=marker, s=100)
        plt.xlabel(axis_title)

In [24]:
def plot_oriented_distortion(distor, orientation, pixel_nb):
    if orientation == "h": 
        distance = distor[0]
        barycentre = distor[1]
        marker = r"$\updownarrow$"
        label = "Horizontal distance"
    elif orientation == "v":
        distance = distor[0]
        barycentre = distor[1]
        marker = r"$\leftrightarrow$"
        label = "Vertical distance"
    mean_dist = pixel_nb
    cm = plt.cm.get_cmap('RdYlBu')
    nb_x, nb_y = distance.shape
    dist_deviation = distance - mean_dist
    graph = plt.scatter(barycentre[0], barycentre[1], c=dist_deviation, cmap=cm,
               marker=marker, s = 100,
               label=label)
    return graph

In [18]:
def get_barycentres(point1, point2):
    x1, y1 = point1[:, :, 0], point1[:, :, 1]
    x2, y2 = point2[:, :, 0], point2[:, :, 1]
    x_bary = (x1+x2)/2
    y_bary = (y1+y2)/2
    return x_bary, y_bary